In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import r2_score, make_scorer
import xgboost as xgb
import optuna

# Load data
train_data = pd.read_csv("/kaggle/input/challenge-1/train.csv")
test_data = pd.read_csv("/kaggle/input/challenge-1/Test_final.csv")

# Store test IDs
test_ids = test_data['ID']

# Drop ID column if present
train_data.drop(columns=['ID'], errors='ignore', inplace=True)
test_data.drop(columns=['ID'], errors='ignore', inplace=True)

# Convert 'Session start time' to numeric and apply sqrt transformation
train_data['Session start time'] = pd.to_numeric(train_data['Session start time'], errors='coerce')
test_data['Session start time'] = pd.to_numeric(test_data['Session start time'], errors='coerce')

train_data['Session start time'] = np.sqrt(train_data['Session start time'])
test_data['Session start time'] = np.sqrt(test_data['Session start time'])

# One-hot encoding for categorical variables
train_data = pd.get_dummies(train_data, drop_first=True)
test_data = pd.get_dummies(test_data, drop_first=True)

# Align test data columns with train data
missing_cols = set(train_data.columns) - set(test_data.columns)
for col in missing_cols:
    test_data[col] = 0

test_data = test_data[train_data.columns.drop('Absenteeism time in hours', errors='ignore')]

# Handle missing values
train_data.fillna(train_data.median(numeric_only=True), inplace=True)
test_data.fillna(test_data.median(numeric_only=True), inplace=True)

# Extract target variable
target_column = 'Absenteeism time in hours'
y = train_data[target_column]
X = train_data.drop(columns=[target_column])

# Scale features using RobustScaler
feature_scaler = RobustScaler()
X_scaled = feature_scaler.fit_transform(X)
test_scaled = feature_scaler.transform(test_data)

# Scale target variable
target_scaler = RobustScaler()
y_scaled = target_scaler.fit_transform(y.values.reshape(-1, 1)).flatten()

# Define objective function for Optuna
kf = KFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 20),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'random_state': 42,
    }
    
    model = xgb.XGBRegressor(**params)
    scores = cross_val_score(model, X_scaled, y_scaled, cv=kf, scoring=make_scorer(r2_score))
    return np.mean(scores)

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=60)

# Best hyperparameters and R² score
best_r2_score = study.best_value
best_params = study.best_params

print(f"Best R² Score: {best_r2_score}")
print(f"Best Hyperparameters: {best_params}")

# Train final model with best hyperparameters
final_model = xgb.XGBRegressor(**best_params)
final_model.fit(X_scaled, y_scaled)

# Predict on test set
test_predictions_scaled = final_model.predict(test_scaled)
test_predictions = target_scaler.inverse_transform(test_predictions_scaled.reshape(-1, 1)).flatten()

# Create submission file
submission = pd.DataFrame({
    "ID": test_ids,
    "Absenteeism time in hours": np.round(test_predictions)
})

# Save submission file
submission.to_csv("chal1.csv", index=False)

print("Submission file saved as 'chal1.csv'.")
print(submission.head())     

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:1533: RuntimeWarning: All-NaN slice encountered
  self.center_ = np.nanmedian(X, axis=0)
/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1384: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:114: RuntimeWarning: invalid value encountered in less
  constant_mask = scale < 10 * np.finfo(scale.dtype).eps
[I 2025-04-03 08:32:39,210] A new study created in memory with name: no-name-bb6bd1aa-74ef-4288-9625-6ca9a38f3573
[I 2025-04-03 08:32:40,132] Trial 0 finished with value: -0.09719314483159133 and parameters: {'n_estimators': 689, 'learning_rate': 0.15612729594843147, 'max_depth': 10, 'subsample': 0.6071619710319465, 'colsample_bytree': 0.8142113222232477, 'gamma': 0.19338431645662635, 'reg_lambda': 5.817562451658207, 'reg_alpha': 3.327007441906206, 'min_child_weight': 10}. Best is trial 0 with value: -0.0

Best R² Score: 0.042813122297961456
Best Hyperparameters: {'n_estimators': 136, 'learning_rate': 0.2972315581273567, 'max_depth': 7, 'subsample': 0.9665389229729809, 'colsample_bytree': 0.5402327418388808, 'gamma': 0.33310434275798645, 'reg_lambda': 11.475854761049114, 'reg_alpha': 9.819754696964889, 'min_child_weight': 1}
Submission file saved as 'chal1.csv'.
        ID  Absenteeism time in hours
0  2201048                        3.0
1  2302569                        5.0
2  2101843                       10.0
3  2101729                        6.0
4  2301185                        6.0
